In [1]:
import numpy as np

import os

In [2]:
"""

    where am i?

"""

PATH = os.getcwd() + '/'
data_path = PATH + 'data/'

print(f'PATH: {PATH}')
print(f'data path: {data_path}')

PATH: /Users/tontoman/Documents/GitHub/pytorch/RNN_character_prediction/
data path: /Users/tontoman/Documents/GitHub/pytorch/RNN_character_prediction/data/


In [3]:
"""

    load text

"""

with open( data_path + 'the_great_gatsby.txt', 'r') as f:
    text = f.read()
    
# some cleaning
text = text.replace('\n', '')
    
n = 100
print(f'\nfirst {n} characters')
text[:100]


first 100 characters


'Project Gutenberg AustraliaTitle:      The Great GatsbyAuthor:     F. Scott Fitzgerald* A Project Gu'

In [4]:
"""

    tokenization and encoding

"""

chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = { ch: v for v, ch in int2char.items() }

# encode the text
encoded = np.array( [ char2int[ch] for ch in text ] )

print('\nencoded text: ')
encoded


encoded text: 


array([54, 23, 28, ...,  0,  4,  7])

In [5]:
"""

    pre-processing the data
    
    the LSTM needs an input that is one-hot encoded meaning that each character is converted to an integer, and then converted to a column vector where only it's corresponding integer index will have the 
    value of 1 and the rest will be filled with 0's.

"""

def one_hot_encoder(arr, n_labels):
    
    # initialize the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # fill the appropriate elements with ones
    one_hot[ np.arange( one_hot.shape[0] ), arr.flatten() ] = 1
    
    # finally reshape it to get back to the original array
    one_hot = one_hot.reshape( ( *arr.shape, n_labels ) )
    
    return one_hot
    
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encoder(test_seq, 8)

print(f'\ntest sequence shape: {test_seq.shape}')
print(f'one hot encoded shape: {one_hot.shape}')
one_hot.shape


test sequence shape: (1, 3)
one hot encoded shape: (1, 3, 8)


(1, 3, 8)

In [14]:
"""

    define the training mini-batches
    
    N <- batch size
    M <- sequence lenght
    K <- number of batches

"""

def get_batches(arr, batch_size, seq_length):
    
    # get the number of nbatches
    total_batch = batch_size * seq_length # N * M
    n_batches = arr.shape[0] // total_batch # k
    
    # get only enough characters to make the batches
    arr = arr[ :n_batches * total_batch ]
    
    # reshape into batch size rows
    size = ( batch_size, -1 ) # -1 is the placeholder
    arr = arr.reshape( size)
    
    # iterate over the batches using a window of size seq_length
    
    for n in range( 0, arr.shape[1] , seq_length):
        
        # features
        x = arr[ :, n : n + seq_length ]
        
        # the targets, shifted by one
        y = np.zeros_like(x)
        
        try:
            
            y[ : , : -1], y[ : , -1 ] = x[ : , 1 : ], arr[ : , n+seq_length ]
            
        except IndexError:
            
            y[ : , : -1 ], y[ : , -1 ] = x[ : , 1 : ], arr[ : , 0 ]
        
        yield x, y
    
batch_size = 2
seq_lenght = 3

test_seq = np.arange(12) 

# test implementation
batches = get_batches( test_seq, batch_size, seq_lenght )
x, y = next(batches)

# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[0 1 2]
 [6 7 8]]

y
 [[1 2 3]
 [7 8 9]]


In [ ]:
"""

    defining the network with pytorch

"""

